In [136]:

%matplotlib inline
import requests
import re
import time
import os
import urllib.request
from typing import Union
import csv
import statistics
import pdf2docx
import csv
import pickle
from openpyxl import load_workbook

import typing
import io
import numpy as np
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
%config Completer.use_jedi = False
from pathlib import Path
# import tqdm.notebook.tqdm as tqdm
from tqdm import tqdm_notebook
%matplotlib widget
import matplotlib.pyplot as plt
import logging
logging.basicConfig(format=u'%(filename)+13s [ LINE:%(lineno)-4s] %(levelname)-8s %(message)s',
                    level=logging.DEBUG)

logger = logging.getLogger(__name__)

from docx2python import docx2python

import xlrd


In [690]:


class DataCleaner:
    """убирает лишние данные"""

    @staticmethod
    def remove_unwanted_symbols(df):        
        # TODO: чистка всех колонок
        df = df.applymap(lambda x: str(x).replace('\n', ' '))
        df = df.applymap(lambda x: ' '.join(str(x).split()))
        
        return df
    
    @staticmethod
    def remove_unwanted_cells(df):
        print('КОЛОНКИ В remove_unwanted_cells ----- ', df.columns)
        
        # убирает ячейки с нумерацией
        # print('--- DataCleaner.remove_unwanted_cells ---', df.columns)
        #TODO: почему тут только должность?

        # df = df[~df['position'].astype(str).str.isdigit()]
        return df
    
    
    @staticmethod
    def remove_unwanted_rows(df: pd.DataFrame):
        headers_pattern = '(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество|\
                рублей|руб|cреднемесячная|зарпл.|плат[ы, а, е]|заработн[ой, ая]|плат[е, а, ы]|\
                    cреднемесячн[ая, ой]|зарплат[а, е, ной, ы]должност[ь, и, ей]|предприяти[е,я]|учреждени[е,я]|юридическое лицо)'

        to_delete = []
        temp_df = df.copy()
        cols_we_need = ['department', 'position', 'name', 'salary']
        temp_df = temp_df[[c for c in temp_df.columns if c in cols_we_need]]
        temp_df = temp_df.applymap(lambda x: str(x).lower())
        temp_df.to_excel('temp_df.xlsx')

        for row in temp_df.itertuples():
            index = row[0]
            headers_in_row = sum([bool(re.search(pattern=headers_pattern, string=str(e))) for e in list(row)])
            if headers_in_row > 2:
                to_delete.append(index)

            if len([e for e in set(row[1:]) if e and len(str(e)) > 2]) < 2:
                to_delete.append(index)

        df.drop(index=to_delete, axis=1, inplace=True)
        return df

    @staticmethod
    def remove_short_cols(df: pd.DataFrame):
        pass
        to_drop = []
        # df = df.applymap(str)
        # for i in len(df.columns):
        #     mean_cell_lenght = statistics.mean(list(map(len, df.iloc[:,i].values)))
        #     if mean_cell_lenght < 4:
        #         to_drop.append(i)
        
        # return df.drop()

    @staticmethod
    def remove_short_rows(df):
        # удаляет ряды с недостаточными данными
        # ! должно применяться после выбора норм колонок
        to_remove = []
        for tup in df.itertuples():
            res = [len(str(e)) for e in tup]
            if statistics.mean(res) < 5:
                to_remove.append(tup.Index)
        
        df.drop(index=to_remove, inplace=True)
        return df

    @staticmethod
    def convert_salary(df: pd.DataFrame):
        def converter(salary):
            pass
    
        pass


    @staticmethod
    def concat_same_cols(df: pd.DataFrame):
        def sjoin(x): return ' '.join(x[x.notnull()].astype(str))
        return df.groupby(level=0, axis=1).apply(lambda x: x.apply(sjoin, axis=1))


    @staticmethod
    def get_numeric_salary(df: pd.DataFrame):
        # иногда в salary попадают колонки с описанием 
        
        cols = list(df.columns)
        if cols.count('salary') <= 1:
            return df

        def drop_not_numeric_salary(df) -> int:
            # берем только salary где большая часть - цифры

            max_val = 0
            realy_salary_col = 0
            for i in range(len(df.columns)):
                
                num_of_numeric_vals = sum(df.iloc[:,i].values)
                if num_of_numeric_vals > max_val:
                    realy_salary_col=i

            return realy_salary_col 
            
        salary_subset = df['salary'].copy()
        bool_salary_subset = salary_subset.applymap(lambda x: str(x).isalnum())
        salary_col = drop_not_numeric_salary(bool_salary_subset)
        salary_subset = salary_subset.iloc[:, salary_col]
#        salary_subset.to_excel('salary_subset.xlsx')
        
        df['salary'] = salary_subset
        df = df.T.groupby(level=0).first().T # удалили дубликаты

              
        return df

    




    def clean_df(self, df):
        df = self.remove_unwanted_symbols(df)
        df = self.get_numeric_salary(df)
        df = self.remove_unwanted_rows(df)
        # df = self.remove_unwanted_cells(df)
        # df = self.remove_short_rows(df)
        # df = self.concat_same_cols(df)
        return df



In [700]:
class ExcelParser:

    def __init__(self):
        self.data_cleaner = DataCleaner()
        self.cols_we_need = ['department', 'position', 'name', 'salary', 'sheet_name', 'documentfile_id']

    @staticmethod
    def find_ok_cols(cols: tuple) -> Union[dict[int, str], None]:
        '''ищем заголовки в таблице'''
        
        cols = list(map(str, cols))[:-1]
        cols = list(map(str.lower, cols))
        if len(set(cols)) == 1:
            return 

        ok_cols = 0
        result = {} # 'n_of_col':'new_name'
        for i, col in enumerate(cols):
            if len(result) > 5 or i > 20:
                return result

            if re.search(pattern='(фамилия|имя|фио|ф\.и\.о\.|ф\.и\.о|отчество)', string=col) and 'name' not in result.values():
                result[i] = 'name'
                ok_cols+=1

            elif re.search(pattern='(рублей|руб|cреднемесячная|зарпл.|плат[ы, е, а]|заработн[ой, ая] плат[а, ы]|cреднемесячн[ая, ой]|зарплат[а, ной, ы])', string=col): #and "salary" not in result.values():
                result[i] = "salary"
                ok_cols+=1
                
            elif re.search(pattern='(должност[ь, и, ей])', string=col) and 'position' not in result.values(): 
                result[i] = 'position'
                ok_cols+=1
                
            elif re.search(pattern='(предприяти[е,я]|учреждени[е,я]|юридическое лицо)', string=col) and 'department' not in result.values():
                result[i] = 'department'
                ok_cols+=1

        if len(result) > 2:
            return result
        return False

        

    def rename_cols(self, df: pd.DataFrame, new_cols: dict[int, str]):
        for k,v in new_cols.items():
            
            # if v == 'salary':
            #     df.rename(columns={df.columns[k]: 'salary_' + df.columns[k] }, inplace = True)
            # else:
            df.rename(columns={df.columns[k]: v}, inplace = True)
        return df 
    
    @staticmethod # table splitter
    def find_department_in_table(df: pd.DataFrame) -> list[dict[int, str]]: # [index, department]
        departments_n_indexes = []
        for row in df.itertuples():
            index = row[0]
            row = list(row)[1:]
                    
            if len(set(row))<2:
                if not all([type(e) in [int, float] for e in row]):
                    if statistics.mean([len(e) for e in row]) > 4:
                        departments_n_indexes.append({'index':index, 'dep': row[0]})
        
        return departments_n_indexes
    
    @staticmethod            
    def add_department_info_to_df(df: pd.DataFrame, dep_info: list[dict[int, str]]) -> pd.DataFrame:
        df['department'] = None
        for data in dep_info: 
            df.at[data['index'], 'department'] = data['dep']       
        df = df.dropna(axis=1, how='all').fillna(method='ffill', axis=0)
        return df


    def detect_headers(self, df: pd.DataFrame)-> pd.DataFrame:
        for row in df.itertuples():
            index = row[0]
            new_cols = self.find_ok_cols(row[1:])

            if new_cols:
                break

        if not new_cols:
            raise ValueError('не нашли заголовки таблицы')
            #return 'не нашли заголовки'


        if 'department' not in new_cols.values(): 
            # если нет колонки с учреждениями - идем искать в таблице 
            dep_info = self.find_department_in_table(df)
            if dep_info:
                df = self.add_department_info_to_df(df, dep_info)
            else:
                merged_cells = ' '.join([e for e in set(list(map(str,sum(list(list(e) for e in df.iloc[:index,:].values),[])))) if 'nan' not in e and 'Unnamed' not in e and not e.isalnum() and e not in self.cols_we_need])
                cols = ' '.join ([e for e in set(list(map(str, list(df.iloc[:index,:].columns)))) if 'nan' not in e and 'Unnamed' not in e and not e.isalnum() and e not in self.cols_we_need])
                df['department'] = cols + ' ' + merged_cells
   
        df = self.rename_cols(df, new_cols)
        
        return df

    
    def open_xlsx(self, filename: Path) -> Union[list[pd.DataFrame], None]:
        wb = load_workbook(folder/  file) 
        sheets = wb.sheetnames
        dfs = []
        for sheet in sheets:
            df = pd.read_excel(filename, sheet_name=sheet, header=None)
            if not df.empty:
                df['sheet_name'] =  sheet
                dfs.append(df)
                
        if not dfs:
            raise ValueError(f'Не нашли таблиц в {filename}')
        return dfs

    def open_xls(self, filename: Path) -> list[pd.DataFrame]:
        wb = xlrd.open_workbook(folder / file, ignore_workbook_corruption=True)
        sheets = wb.sheet_names()
        dfs = []

        for sheet in sheets:
            df = pd.read_excel(filename, sheet_name=sheet, header=None)
            if not df.empty:
                df['sheet_name'] =  sheet
                dfs.append(df)

        if not dfs:
            raise ValueError(f'Не нашли таблиц в {filename}')
        return dfs
    
    @staticmethod
    def salary_parser(salary:str)->int:
        salary = str(salary)
        salary = salary.split(',')[0]
        salary = salary.split('.')[0]
        salary = salary.replace(' ', '')
        salary = ''.join([c for c in salary if c.isdigit()])

        if salary.isalnum():    
            return int(salary) * 12 

        return salary
    


    def parse_file(self, filename: Path):
        
        try:
            if file.endswith('.xlsx'):
                dfs = self.open_xlsx(filename)
            elif file.endswith('.xls'):        
                dfs = self.open_xls(filename)
            else:
                raise ValueError('Расширение файла должно быть .xlsx или .xls')
        
        except Exception as ex:
            print(ex)

        parsed_dfs = []
        
        for df in dfs:
            df = df.dropna(axis=1, how='all').fillna(method='ffill', axis=1)                

            df = self.detect_headers(df)
            
            df = df.fillna(method='ffill', axis=0)


            df = df[[c for c in df.columns if c in self.cols_we_need]]
            df = self.data_cleaner.clean_df(df)

            # слить имена 
            #df = df.T.groupby(level=0).first().T # удалили дубликаты
        
            '''def sjoin(x): return ' '.join(x[x.notnull()].astype(str))
            return df.groupby(level=0, axis=1).apply(lambda x: x.apply(sjoin, axis=1))
            '''
            if 'salary' in df.columns:
                # TODO: понять что делать с более чем одной зарплатой
                df['salary_raw'] = df['salary']
                df['salary'] = df['salary'].apply(lambda x: self.salary_parser(x))
            
            df['documentfile_id'] = filename.name.split('_')[0]
            parsed_dfs.append(df)

        return parsed_dfs    
        # try:
        #     return pd.concat(parsed_dfs)
        # except:
        #     return parsed_dfs



# TODO: 
# 1. дубликаты
# 2. убрать ряды со служебной инфой
# клинер тут же. потому что так проще конкатинировать  

file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx\88439_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xlsx"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx\83293_2019_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xlsx"
# file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx\83290_2017_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xlsx"?
#file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xls\83352_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xls"
file = r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx\105387_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii.xlsx"
excel_parser = ExcelParser()
# res = excel_parser.parse_file(Path(file))

my_dic = {}
my_dic['documents'] = []

my_dic['persons'] = []
my_dic['document'] = {}


# for e in res.itertuples():
#     person = {
#         'person':{
#             'name': e.name,
#             'role': e.position,
#             'department' : e.department,
#             'incomes': e.salary,
#             'raw_income': e.salary_raw
#         }
#     }

#     my_dic['persons'].append(person)

# my_dic    
# res = excel_parser.parse_file(Path(file))
res[2]

ВСЕ РАБОТАЕТ! 
ПЕРЕВЕСТИ В JSON
ПРИКРУТИТЬ КЛИНЕР ОТСЮДА К ПДФ, РАСПАРИТЬ, 




,department,name,position,salary,sheet_name,salary_raw,documentfile_id
1,Лист1 (3) Наименование предприятия: Федерально...,и главных бухгалтеров федеральных государствен...,и главных бухгалтеров федеральных государствен...,24228,Лист1 (3),и главных бухгалтеров федеральных государствен...,105387
2,Лист1 (3) Наименование предприятия: Федерально...,и главных бухгалтеров федеральных государствен...,и главных бухгалтеров федеральных государствен...,24228,Лист1 (3),и главных бухгалтеров федеральных государствен...,105387
5,Лист1 (3) Наименование предприятия: Федерально...,Милькаманович А.А.,И.о.генерального директора,,Лист1 (3),"руководителя, руб.",105387
6,Лист1 (3) Наименование предприятия: Федерально...,Иванов В.А.,Первый заместитель генерального директора,,Лист1 (3),"заместителя руководителя, руб.",105387
7,Лист1 (3) Наименование предприятия: Федерально...,Ли Ю.Ч.,Заместитель генерального директора по правовым...,,Лист1 (3),"заместителя руководителя, руб.",105387
8,Лист1 (3) Наименование предприятия: Федерально...,Пугачева Д.В.,Заместитель генерального директора по финансам,,Лист1 (3),"заместителя руководителя, руб.",105387
9,Лист1 (3) Наименование предприятия: Федерально...,Забровская А.А.,Главный бухгалтер,,Лист1 (3),"главного бухгалтера, руб.",105387
10,Лист1 (3) Наименование предприятия: Федерально...,Базенко Е.М.,Директор филиала,,Лист1 (3),"Директора филиала, руб.",105387
11,Лист1 (3) Наименование предприятия: Федерально...,Мокина М.В.,Главный бухгалтер филиала,,Лист1 (3),"Главного бухгалтера филиала, руб.",105387


,department,name,position,salary,sheet_name,salary_raw,documentfile_id
1,Информация о расчитываемой за календарной год ...,и главных бухгалтеров федеральных государствен...,и главных бухгалтеров федеральных государствен...,24228,Лист1,и главных бухгалтеров федеральных государствен...,105387
2,Информация о расчитываемой за календарной год ...,и главных бухгалтеров федеральных государствен...,и главных бухгалтеров федеральных государствен...,24228,Лист1,и главных бухгалтеров федеральных государствен...,105387
5,Информация о расчитываемой за календарной год ...,Тихолиз Михаил Владимирович,Генеральный директор,1770228,Лист1,147519,105387
6,Информация о расчитываемой за календарной год ...,Чеботарева Марина Сергеевна/ Зайцева Екатерина...,Первый заместитель генерального директора,1566420,Лист1,130535,105387
7,Информация о расчитываемой за календарной год ...,Зайцева Екатерина Дмитриевна,Заместитель генерального директора по взаимоде...,1549092,Лист1,129091,105387
8,Информация о расчитываемой за календарной год ...,Гайдина Екатерина Евгеньевна,Заместитель генерального директора по операцио...,1194996,Лист1,99583,105387
9,Информация о расчитываемой за календарной год ...,Семёнова Мария Валерьевна,Главный бухгалтер,1680000,Лист1,140000,105387


In [438]:
d = {
  "properties": {
    "persons": {
      "$id": "/properties/persons",
      "type": "array",
      "items": {
        "type": "object",
        "properties": {
          "person": {
            "$id": "/properties/persons/properties/person",
            "type": "object",
            "properties": {
              "name": {
                "$id": "/properties/persons/properties/person/properties/name",
                "type": "string",
                "title": "Фамилия Имя Отчество (или с формате \"Фамилия И.О.\")",
                "default": "",
                "examples": [
                  "Бродский Михаил Наумович",
                  "Бродский М. Н.",
                  "Бродский М.Н."
                ]
              },
              "name_raw": {
                "$id": "/properties/persons/properties/person/properties/name_raw",
                "type": "string",
                "title": "Фамилия Имя Отчество (сырые данные из документа, указывать точно как было)",
                "default": "",
                "examples": [
                  "Бродский МихаиллНаумович",
                  "Бродский М...Н.",
                  "БродскийМ.Н."
                ]
              },
              "role": {
                "$id": "/properties/persons/properties/person/properties/role",
                "title": "Должность",
                "items": {
                  "examples": [
                    "Представитель Губернатора Санкт-Петербурга в Законодательном Собрании Санкт-Петербурга"
                  ]
                }
              },
              "department": {
                "$id": "/properties/persons/properties/person/properties/department",
                "title": "Департамент в органе власти",
                "examples": [
                  "Отдел по борьбе с коррупцией"
                ]
              }
            },
          },
          "incomes": {
            "$id": "/properties/persons/properties/incomes",
            "items": {
              "$id": "/properties/persons/properties/incomes/items",
              "type": "object",
              "properties": {
                "size": {
                  "$id": "/properties/persons/properties/incomes/items/properties/size",
                  "type": [
                    "number",
                    "null"
                  ],
                  "title": "Объём дохода в рублях",
                  "default": 0,
                  "examples": [
                    1474895
                  ]
                },
                "comment": {
                  "$id": "/properties/persons/properties/incomes/items/properties/comment",
                  "type": [
                    "string",
                    "null"
                  ],
                },
                },
            
          },

          "document_position": {
            "$id": "/properties/persons/properties/document_position",
            "type": "string",
            "title": "Адрес ячейки XLSX, номер страницы DOCX или другая информация для поиска записи в оригинальном документе"
          },
          "person_index": {
            "$id": "/properties/persons/properties/person_index",
            "type": "number",
            "title": "Порядковый номер в данном документе (если был указан в отдельном столбце)"
          }
        },
        "required": [
          "person"
        ]
      }
    },


    "document": {
      "$id": "/properties/document",
      "type": "object",
      "properties": {
        "documentfile_id": {
          "$id": "/properties/document/properties/documentfile_id",
          "type": ["number", "null"],
          "title": "ID файла в базе Декларатора",
          "default": "null",
          "examples": [
            35304
          ]
        },
        "archive_file": {
          "$id": "/properties/document/properties/archive_file",
          "type": "string",
          "title": "Полный путь к данном файлу внутри архива",
          "description": "Только для DocumentFile типа zip, где было несколько файлов внутри",
          "examples": [
            "Сотрудники министерства.xlsx",
            "руководители/2013/уточненный.docx"
          ]
        },
        "sheet_number": {
          "$id": "/properties/document/properties/sheet_number",
          "type": "number",
          "title": "Индекс листа (только для Excel)",
          "examples": [
            0, 1, 3
          ]
        },
        "sheet_title": {
          "$id": "/properties/document/properties/sheet_title",
          "type": "string",
          "title": "Заголовок листа (только для Excel)",
          "description": "Используется в случае, если где-то в шапке есть наименование лиц перечисленных на данном листе",
          "default": "null",
          "examples": [
            "Сведения о доходах, расходах, об имуществе и обязательствах имущественного характера, представленных федеральными государственными гражданскими служащими Министерства Российской Федерации по развитию Дальнего Востока, за период с 1 января 2015 г. по 31 декабря 2015 г."
          ]
        },
        "type": {
          "$id": "/properties/document/properties/type",
          "type": "string",
          "title": "Тип декларации",
          "default": "",
          "examples": [
            "Антикоррупционная декларация"
          ]
        },
        "name": {
          "$id": "/properties/document/properties/name",
          "type": "string",
          "title": "Название документа",
          "default": "",
          "examples": [
            "Сведения о доходах, об имуществе и обязательствах имущественного характера Губернатора Санкт-Петербурга, а также его супруги за период с 1 января 2016 года по 31 декабря 2016 года"
          ]
        },
        "file_size": {
          "$id": "/properties/document/properties/file_size",
          "type": "number",
          "title": "Размер файла документа",
          "default": "null"
        },
      
        "year": {
          "$id": "/properties/document/properties/year",
          "type": ["integer", "null"],
          "title": "Год декларируемых доходов",
   },
      
      
      }


    }
  }
}
}








In [648]:
t = pd.read_excel(r"D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx\105387_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii.xlsx")
vals = list(t.iloc[:3,:].columns) + list(t.iloc[:3,:].values)
vals

merged_cells = ' '.join([e for e in list(map(str,sum(list(list(e) for e in t.iloc[:3,:].values),[]))) if 'nan' not in e and 'Unnamed' not in e])
cols = ' '.join ([e for e in list(map(str, list(t.iloc[:3,:].columns))) if 'nan' not in e and 'Unnamed' not in e])

merged_cells + cols


['Информация о расчитываемой за календарной год среднемесячной заработной плате руководителей, их заместителей']

False

In [402]:
folders = [Path(r'D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xlsx'), 
Path(r'D:\PROGR\LEARN_PYTHON\Declarator\declarations-parser\data_ids\xls')]

errors = []
for folder in folders:
    for file in os.listdir(folder):
        try:
            res = excel_parser.parse_file(folder / file)
            
            if type(res) == pd.DataFrame:
                res.to_excel('cool_excel/' + file + '.xlsx')
            elif type(res) == list:
                for e in enumerate(res):
                    e.to_excel(f'cool_excel/{e}__' + e + '.xlsx')
                    
        except Exception as ex:
            errors.append({'file':file, 'error':ex})
            #print(ex)

    

#             res = excel_parser.parse_file(folder / file)
#         except Exception as ex:
#             print(file)
#             print(ex)
        #res.to_excel(f'cool_excel/{file}.xlsx')

            

C:\Users\Kirill\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")



_locate_stream(Workbook): seen
    0  5 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 
   20  4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 2 
Workbook corruption: seen[2] == 4


In [403]:
errors

[{'file': '100576_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii.xlsx',
  'error': ValueError('Wrong number of items passed 4, placement implies 1')},
 {'file': '105387_2019_Rukovoditeli_podvedomstvennykh_uchrezhdenii.xlsx',
  'error': ValueError('Wrong number of items passed 4, placement implies 1')},
 {'file': '83297_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xlsx',
  'error': ValueError('Wrong number of items passed 4, placement implies 1')},
 {'file': '88338_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xlsx',
  'error': ValueError('Wrong number of items passed 4, placement implies 1')},
 {'file': '88439_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xlsx',
  'error': ValueError('не нашли заголовки таблицы')},
 {'file': '83352_2018_Rukovoditeli,_zamestiteli_i_glavnye_bukhgaltery_podvedomstvennykh_uchrezhdenii.xls',
  'error': UnboundLocalError("local variable 'dfs' 

True